<a href="https://colab.research.google.com/github/kkruglik/female_streets/blob/main/russian_streets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Обработка данных

In [ ]:
pip install pymorphy2 pymorphy2-dicts DAWG-Python pandarallel -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 107.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 119.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/Adress_klassificator_FIAS+Pochta_RF_179_14.10.21.zip" -d .

Archive:  /content/drive/MyDrive/Colab Notebooks/Adress_klassificator_FIAS+Pochta_RF_179_14.10.21.zip
  inflating: ./administrative_addresses.csv  
  inflating: ./meta.csv              
  inflating: ./structure.csv         


In [ ]:
import requests
import json
import pandas as pd
import pymorphy2
from pandarallel import pandarallel
import numpy as np

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
pandarallel.initialize(progress_bar=True, nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
male_names = [
    'август', 'агап', 'агафон', 'адам', 'адриан', 'азарий', 'аким', 'алан', 'александр', 'алексей',
    'альберт', 'анатолий', 'андрей', 'антип', 'антон', 'анфим', 'аполлинарий', 'арий', 'аристарх',
    'аркадий', 'арно', 'арнольд', 'арсений', 'артем', 'артемий', 'артур', 'архип', 'афанасий',
    'боголюб', 'болеслав', 'борис', 'борислав', 'бронислав', 'вадим', 'валентин', 'валерий',
    'вальдемар', 'варфоломей', 'василий', 'ватслав', 'велизар', 'венедикт', 'вениамин', 'викентий',
    'виктор', 'вилен', 'виссарион', 'виталий', 'владимир', 'владислав', 'владлен', 'власий', 'володар',
    'всеволод', 'вячеслав', 'гектор', 'геннадий', 'георгий', 'герасим', 'герман', 'глеб', 'горислав',
    'григорий', 'даниил', 'данислав', 'демид', 'демьян', 'денис', 'дмитрий', 'добрыня', 'дорофей',
    'евгений', 'евграф', 'евдоким', 'евсей', 'евстафий', 'егор', 'елизар', 'елисей', 'емельян',
    'еремей', 'ермак', 'ермил', 'ермолай', 'ерофей', 'ефим', 'ефрем', 'жан', 'захар', 'зигмунд',
    'зиновий', 'зосима', 'зураб', 'иван', 'игнат', 'игнатий', 'игорь', 'илларион', 'илья', 'инокентий',
    'ипполит', 'казимир', 'кай', 'камиль', 'ким', 'кир', 'кирилл', 'клавдий', 'клим', 'кондрат',
    'константин', 'корней', 'корнелий', 'кузьма', 'лавр', 'лаврентий', 'лазарь', 'ларион', 'лев',
    'леонард', 'леонид', 'леонтий', 'лука', 'лукьян', 'любомир', 'макар', 'макарий', 'максим',
    'максимильян', 'марк', 'маркел', 'марсель', 'мартин', 'матвей', 'мечеслав', 'микула', 'милан',
    'мирон', 'мирослав', 'митрофан', 'михаил', 'модест', 'мстислав', 'нестор', 'никанор', 'никита',
    'никифор', 'никодим', 'никола', 'николай', 'олег', 'онисим', 'орест', 'осип', 'остап', 'павел',
    'панкрат', 'парамон', 'пахом', 'петр', 'порфирий', 'потап', 'прокофий', 'прохор', 'радомир',
    'радослав', 'рафаэль', 'ренат', 'роберт', 'родион', 'ролан', 'роман', 'ростислав', 'рудольф',
    'савелий', 'светозар', 'святослав', 'семен', 'серафим', 'сергей', 'спартак', 'спиридон',
    'станислав', 'степан', 'стефан', 'тарас', 'терентий', 'тимофей', 'тихомир', 'тихон', 'трифон',
    'трофим', 'ульян', 'устин', 'федор', 'федот', 'феликс', 'феофан', 'филарет', 'филат', 'филимон',
    'филипп', 'фирс', 'фома', 'фрол', 'харитон', 'эдуард', 'эмилий', 'эраст', 'эрик', 'эрнест', 'юлиан',
    'юлий', 'юрий', 'юст', 'яков', 'ян', 'януарий', 'яромир', 'ярослав'
                    ]
                    
female_names = [
    'августа', 'авдотья', 'аврора', 'агата', 'аглая', 'агнесса', 'агния', 'ада', 'адель', 'аза',
    'азалия', 'аида', 'аксинья', 'акулина', 'алевтина', 'александра', 'алексия', 'алёна', 'алена', 'алина',
    'алиса', 'алла', 'альберта', 'амалия', 'анастасия', 'ангелина', 'андриана', 'анеля', 'анжела',
    'анжелика', 'анисья', 'анита', 'анна', 'антонина', 'анфиса', 'апрелия', 'ариадна', 'арина',
    'арсения', 'ассоль', 'астра', 'ася', 'аурика', 'аэлита', 'бажена', 'беата', 'беатриса', 'белла',
    'беляна', 'береслава', 'берта', 'богдана', 'божена', 'бронислава', 'валентина', 'валерия',
    'ванда', 'ванесса', 'варвара', 'василина', 'василиса', 'васса', 'веда', 'венера', 'вера',
    'вероника', 'веселина', 'веста', 'вета', 'вивиана', 'видана', 'виктория', 'вилора', 'виола',
    'виринея', 'виталия', 'влада', 'владилена', 'владимира', 'владислава', 'власта', 'воля',
    'всеслава', 'габриэлла', 'гайя', 'галина', 'гелия', 'гелла', 'гера', 'герда', 'глафира', 'глория',
    'грация', 'грета', 'дайна', 'дана', 'даниэла', 'дарина', 'дарья', 'дарьяна', 'декабрина', 'дея',
    'дженни', 'джулия', 'диана', 'дина', 'добрава', 'доля', 'доминика', 'дорофея', 'ева', 'евгения',
    'евдокия', 'екатерина', 'елена', 'елизавета', 'емельяна', 'есения', 'ефимия', 'ефросиния',
    'жанна', 'жасмин', 'ждана', 'забава', 'залина', 'зара', 'зарема', 'зарина', 'звана', 'звенислава',
    'земфира', 'зинаида', 'зиновия', 'зита', 'злата', 'златослава', 'зоряна', 'зоя', 'иванна',
    'ивона', 'ида', 'изабелла', 'изольда', 'илона', 'иля', 'инга', 'инесса', 'инна', 'иоанна',
    'иоланта', 'ирина', 'ирма', 'искра', 'июлия', 'камилла', 'карина', 'каролина', 'катарина', 'кира',
    'клавдия', 'клара', 'кристина', 'ксения', 'лада', 'лана', 'лара', 'лариса', 'лаура', 'леля',
    'лера', 'леся', 'лиана', 'лидия', 'лика', 'лилиана', 'лилия', 'лина', 'линда', 'лира', 'лия',
    'лолита', 'лора', 'луиза', 'любава', 'любовь', 'людмила', 'майя', 'малика', 'мальвина',
    'маргарита', 'марианна', 'марина', 'мария', 'марта', 'матрена', 'мелания', 'мелисса', 'мила',
    'милада', 'милана', 'милолика', 'милослава', 'мира', 'мирослава', 'мишель', 'мия', 'млада',
    'моника', 'муза', 'надежда', 'нана', 'наталья', 'нева', 'нелли', 'ника', 'николина', 'николь',
    'нила', 'нина', 'нинель', 'нона', 'оксана', 'олеся', 'олимпия', 'ольга', 'пелагея', 'полина',
    'прасковья', 'рада', 'раиса', 'регина', 'рената', 'риана', 'римма', 'рита', 'роберта', 'рогнеда',
    'роза', 'роксана', 'ростислава', 'рузана', 'руслана', 'руфина', 'сабрина', 'сандра', 'светлана',
    'святослава', 'северина', 'селена', 'серафима', 'сильвия', 'слава', 'славяна', 'снежана', 'соня',
    'софия', 'станислава', 'стелла', 'стефания', 'таисия', 'тамара', 'татьяна', 'ульяна', 'урсула',
    'услада', 'устина', 'фаина', 'феба', 'фекла', 'фрида', 'хана', 'хельга', 'хлоя', 'христина',
    'царина', 'цветана', 'цецилия', 'чулпан', 'шанель', 'шарлотта', 'шейла', 'эвелина', 'эдита',
    'элеонора', 'элина', 'элла', 'элоиза', 'эльвира', 'эльза', 'эльмира', 'эмилия', 'эмма', 'эрика',
    'эсмеральда', 'эстер', 'эшли', 'юлиана', 'юлия', 'юна', 'юния', 'юнона', 'ядвига', 'яна', 'янина',
    'ярина', 'яромира', 'ярослава', 'ясмина', 'фатиме'
                      ]

In [ ]:
# кол-во уникальных женских имен
len(set(female_names))

309

In [ ]:
# кол-во уникальных мужских имен
len(set(male_names))

220

In [ ]:
def get_gender(name):
    count = {'Female':0, 'Male':0, 'Unk':0}
    name_tokens = name.split(' ')

    # переводим токены в именительный падеж    
    norm_names = []
    for n in name_tokens:
        n = morph.parse(n)[0]
        n = n.normal_form
        norm_names.append(n)

    # проверяем есть ли токены в словарях с муж. и жен. именами
    for n in norm_names:
        n = n.lower().strip().replace('ё', 'е')
        if n in female_names:
            count['Female'] += 1
        
        if n in male_names:
            count['Male'] += 1
            
    if count["Female"] > count["Male"]:
        return "Female"
    elif count["Female"] < count["Male"]:
        return "Male"
    return "Unk"          

In [ ]:
# female_names = pd.read_csv(
#     "https://raw.githubusercontent.com/sorokinpf/russian_names/master/russian_female_names.csv",
#     names=['name', 'count'], 
#     header=None)


# male_names = pd.read_csv(
#     "https://raw.githubusercontent.com/sorokinpf/russian_names/master/russian_male_names.csv",
#     names=['name', 'count'], 
#     header=None)


# male_names = male_names.name.to_list()
# female_names = female_names.name.to_list()

In [ ]:
def get_gender(name):
    count = {'Female':0, 'Male':0, 'Unk':0}
    name_tokens = name.split(' ')

    # переводим токены в именительный падеж    
    norm_names = []
    for n in name_tokens:
        n = morph.parse(n)[0]
        n = n.normal_form
        norm_names.append(n)

    # проверяем есть ли токены в словарях с муж. и жен. именами
    for n in norm_names:
        n = n.lower().strip().replace('ё', 'е')
        if n in female_names:
            count['Female'] += 1
        
        if n in male_names:
            count['Male'] += 1
            
    if count["Female"] > count["Male"]:
        return "Female"
    elif count["Female"] < count["Male"]:
        return "Male"
    return "Unk" 

In [ ]:
# загрузка датасета
df = pd.read_csv("administrative_addresses.csv", sep=";")
df = df[df['street'].notnull()]

# убираем ненужные признаки
df = df.filter(items=['region', 'settlement', 'street', 'street_type'])

# оставляем только уникальные улицы для каждого населенного пункта в России
df = df.drop_duplicates(subset=['settlement', 'street'])
df.shape

<ipython-input-44-3a13cfc98984>:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("administrative_addresses.csv", sep=";")


(810334, 4)

In [ ]:
# определяем мужская или женская улица
df["genders"] = df.street.parallel_apply(lambda x: get_gender(x))

In [ ]:
df = df[df.genders != "Unk"]

In [ ]:
df.street = df.street.str.lower()
df.street = df.street.str.strip()
df.street = df.street.str.replace('\d+', '')

<ipython-input-180-2508825ad488>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.street = df.street.str.lower()
<ipython-input-180-2508825ad488>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.street = df.street.str.strip()
<ipython-input-180-2508825ad488>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df.street = df.street.str.replace('\d+', '')
<ipython-input-180-2508825ad488>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

## 2. Отчистка

В женские улицы попадают нерелевантные названия. Например, улица Надежды, Веры, Любви и так далее. С одной стороны это действительно женское имя, но названы эти улицы не честь женщин. 

In [ ]:
remove_names = [
    "надежды",
    "славы",
    "роз",
    "искра",
    "надежда",
    "воинской славы",
    "чулпан",
    "алых роз",
    "народной воли",
    "трудовой славы",
    "веры",
    "любви",
    "шахтерской славы",
    "материнской славы",
    "светлых надежд",
    "нева",
    "искры",
    "надежд",
    "красных роз",
    "боевой славы",
    "сдт виктория",
    "аврора",
    "лилий",
    "виктория",
    "виктории",
    "воли",
    "воля",
    "снт надежда",
    "ратной славы",
    "роза",
    "роза ветров",
    "бульвар роз",
    "ким",
    "кима",
    "5 августа",
    "аллея спортивной славы",
    "славы 1-й",
    "славы -й"
]

In [ ]:
male_streets = [
    "валентина эмирова",
    "валентина берестова",
    "гере гаджитаев",
    "малика джаналиева"
]

In [ ]:
df.loc[df.street.isin(remove_names), "genders"] = "Unk"

In [ ]:
df.loc[df.street.isin(male_streets), "genders"] = "Male"

In [ ]:
df = df[df.genders != "Unk"]

## 3. Исследование

**Какой датасет мы анализируем**

Набор содержит объединённые из двух источников (ФИАС и обработанные порталом kodifikant.ru данные Почты России) географические данные РФ с детализацией до дома.

Цель датасета — создание единого максимально полного классификатора географических данных, с помощью которого можно было бы осуществлять поиск и привязку географических объектов к официальным стандартизированным геоданным. Для осуществления этой привязки можно использовать библиотеку, доступную по ссылке.

Единица наблюдения в датасете — адрес объекта недвижимости. Данные для каждого наблюдения приведены по следующим атрибутам: федеральный округ, субъект РФ, муниципальное образование, населенный пункт, локация, улица, дом.

Набор данных охватывает все регионы Российской Федерации. Всего набор содержит **25 569 964 наблюдений** по 11 атрибутам.

[Источник и описание](https://www.data-in.ru/data-catalog/datasets/179/)

В изначальном датасете больше 20 млн строк. Такое количество данных получается из-за высокой детализации данных, вплоть до дома. Нам для анализа такая детализация не нужна. 

Поэтому оставим только уникальные названия улиц для каждого населённого пункта России. Таким образом у нас остаётся порядка 800 тысяч значений, в которых мы и будем искать женские и мужские улицы.

Пример нашего датасета:

In [ ]:
df.sample(5)

,region,settlement,street,street_type
17198859,Чувашская,Чепкас-Ильметево,Речная,улица
12281880,Адыгея,Яблоновский,Пархоменко,улица
16494596,Саха /Якутия/,Верхневилюйск,Короленко,улица
20135918,Ивановская,Шубино,Цветочная,улица
2095893,Пермский,Пермь,Водолазная,улица


Всего в нашем датасете получилось **1546 женских** и **7719 мужских** улицы. Мужских улиц почти в 4 раза больше.

И наверное их было бы ещё больше. Но мужские и женские улицы мы ищем по наличию имён в названиях. Соотвественно улица Ленина в топ мужских улиц не попадает, потому что в ней нет самого имени.

In [ ]:
# сколько мужских и женских улиц
df.groupby('genders').size()

genders
Female    1546
Male      7719
dtype: int64

В топе наиболее распространённых улиц без особых сюрпризов. Самая часто встречающая женская улица названы в честь Розы Люксембург и Клары Цеткин - революционерок с именами которых связан 8 марта. А также улицы в честь Зои Космодемьянской - первая женщина, удостоенная звания Героя Советского Союза во время Великой Отечественной войны.

In [207]:
# самая распространнённая женская улица
df[df.genders == "Female"] \
    .groupby("street").size().sort_values(ascending=False).head(10).to_frame()

,0
street,
розы люксембург,263
зои космодемьянской,238
клары цеткин,104
полины осипенко,50
марины цветаевой,23
марины расковой,23
валентины терешковой,15
анны ахматовой,13
любови шевцовой,11


Самый популярая мужчиной стал Степан Разин

In [208]:
# самая распространнённая мужская улица
df[df.genders == "Male"] \
    .groupby("street").size().sort_values(ascending=False).head(10).to_frame()

,0
street,
степана разина,428
максима горького,401
льва толстого,308
олега кошевого,248
демьяна бедного,167
спартака,155
ермака,128
сергея лазо,86
александра невского,84


Самый профеминисткий город в плане названий стал Волгоград и Новроссийск. Там больше 20 различных улиц названных в честь женщин.

In [209]:
# в каком городе больше всего женских улиц
df[df.genders == "Female"] \
    .groupby(['settlement']).size().sort_values(ascending=False).head(10).to_frame()

,0
settlement,
Волгоград,26
Новороссийск,20
Краснодар,15
Тверь,15
Уфа,10
Тюмень,10
Киров,9
Керчь,9
Йошкар-Ола,9


Что это за улицы на примере Волгограда:

In [211]:
df[(df.settlement == "Волгоград") & (df.genders == "Female")] \
    .groupby(["street"]).size().sort_values(ascending=False).to_frame()

,0
street,
им валентины терешковой,1
им елены колесовой,1
им. полины осипенко,1
им. паши ангелиной,1
им. ольги форш,1
им. ольги ковалевой,1
им. надежды крупской,1
им. марины расковой,1
им. маргариты агашиной,1


Самым мужскими городами оказались Тюмень и Краснодар:

In [212]:
#в каком городе больше всего мужских улиц
df[df.genders == "Male"] \
    .groupby(['settlement']).size().sort_values(ascending=False).head(10).to_frame()

,0
settlement,
Тюмень,118
Краснодар,113
Волгоград,92
Бугульма,83
Бийск,72
Ижевск,66
Новосибирск,60
Казань,58
Уфа,53


При этом если смотреть по региону, то в целом Краснодарский край лидер по количеству как женских улиц:

In [213]:
# в каком регионе больше всего женских улиц
df[df.genders == "Female"] \
    .groupby(['region']).size().sort_values(ascending=False).head(10).to_frame()

,0
region,
Краснодарский,122
Свердловская,101
Пермский,66
Челябинская,60
Московская,52
Ростовская,49
Татарстан,40
Дагестан,38
Крым,36


Так и мужских улиц:

In [214]:
# в каком регионе больше всего мужских улиц
df[df.genders == "Male"] \
    .groupby(['region']).size().sort_values(ascending=False).head(10).to_frame()

,0
region,
Краснодарский,428
Свердловская,371
Татарстан,297
Саха /Якутия/,278
Башкортостан,264
Дагестан,222
Челябинская,221
Московская,216
Ростовская,208


Алтай, Ингушетия и Кабардино-Балкария оказались регионами всего с одной женской улицой:

In [215]:
df[df.genders == "Female"] \
    .groupby(['region']).size().sort_values(ascending=False).tail(10).to_frame()

,0
region,
Адыгея,4
Камчатский,3
Карелия,3
Амурская,3
Еврейская,2
Чукотский,2
Ямало-Ненецкий,2
Ингушетия,1
Кабардино-Балкарская,1


Ну и посмотрим какая женская улица наиболее популярна в каждом регионе России. Может быть тут будут какие-то региональные особенности.

In [ ]:
pd.set_option('display.max_rows', 100)
grouped = df[df.genders == "Female"].groupby(["region", "street"], as_index=False).size()
idx = grouped.groupby("region")["size"].idxmax()
grouped.loc[idx]

,region,street,size
0,Адыгея,имени зои космодемьянской,1
4,Алтай,евдокии мекшило,1
9,Алтайский,розы люксембург,6
11,Амурская,зои космодемьянской,1
22,Архангельская,розы шаниной,5
28,Астраханская,зои космодемьянской,8
51,Башкортостан,зои космодемьянской,3
70,Белгородская,марии ушаковой,2
80,Брянская,зои космодемьянской,8
93,Бурятия,елены волковой,1


**Что ещё можно сделать?**
- добавить к именам классификацию. Например, поэт или советский революционер, чтобы посмотреть кто из них преобладает в названиях улиц.
- добавить жизни к имена, чтобы посмотреть к какому периоду чаще отсылают названия улиц

**Гипотезы:**
- Женские имена в названиях улиц представлены значительно меньше, чем мужские.
- Использование женских имен в названиях улиц варьируется в зависимости от региона и зависит от культурных, исторических и социальных факторов
- Женские имена с большей вероятностью будут использоваться в названиях улиц в районах с более высоким процентом женского населения.
- Женские имена чаще используются в названиях улиц в городах с большим числом женщин-политических лидеров или там, где исторически были заметны движения за права женщин.
- Женские имена в названиях улиц чаще используются для жилых улиц, чем для крупных дорог и магистралей.